# Update Dataframe

This script has the objective to add submission and comment to the reddit dataframe

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import praw

In [2]:
reddit = praw.Reddit(client_id='2YvrfPL-6r2E7g',
                     client_secret='ClHTUejD1prIgWThqC5-IwswkM4',
                     user_agent='alexparentt')

# Import CSV's into pandas dataframes

In [3]:
df_submissions = pd.read_csv("reddit_submissions_wUTC_2018-05-16_2141.csv",
skipinitialspace=True,
header = 0,
sep = ',',
encoding='latin1')
print(df_submissions.shape)
df_submissions = df_submissions.reset_index() 
del df_submissions['index']

df_comments = pd.read_csv("reddit_comments_wUTC_2018-05-16_2141.csv",
skipinitialspace=True,
header = 0,
sep = ',',
encoding='latin1')
print(df_comments.shape)
df_comments = df_comments.reset_index()
del df_comments['index']

df_submissions = df_submissions.rename(columns={'id':'id'})
df_comments = df_comments.rename(columns={'id':'comment_id'})

(1505, 26)
(7555, 20)


In [4]:
df_submissions = df_submissions.drop(['Unnamed: 0'], axis=1)
df_comments = df_comments.drop(['Unnamed: 0'], axis=1)
print(df_submissions.shape); print(df_submissions.columns); 
print(df_submissions.dtypes)
#print(df_submissions[0:5])
print(type(df_submissions.index))

print(df_comments.shape); print(df_comments.columns)
print(df_comments.dtypes)
#print(df_comments[0:5])
print(type(df_comments.index))

(1505, 25)
Index(['id', 'subreddit_id', 'title', 'score', 'author', 'author_flair_text',
       'likes', 'archived', 'is_video', 'name', 'over_18', 'pinned',
       'quarantine', 'selftext', 'send_replies', 'num_comments',
       'num_crossposts', 'distinguished', 'created', 'created_utc',
       'comment_limit', 'approved_by', 'ups', 'downs', 'gilded'],
      dtype='object')
id                    object
subreddit_id          object
title                 object
score                  int64
author                object
author_flair_text     object
likes                float64
archived                bool
is_video                bool
name                  object
over_18                 bool
pinned                  bool
quarantine              bool
selftext              object
send_replies            bool
num_comments           int64
num_crossposts         int64
distinguished         object
created              float64
created_utc          float64
comment_limit          int64
approved_by 

# Get new data points

In [5]:
attributes_subm = ['id', 
                   'subreddit_id',
              'title', 
              'score', 
              'author',
             'author_flair_text',
              'likes', 
              'archived',
             'is_video',
             'name',
             'over_18',
             'pinned',    
             'quarantine',
             'selftext',
             'send_replies',
             'num_comments',
             'num_crossposts',
             'distinguished',
             'created',
                   'created_utc',
             'comment_limit',
             'approved_by',
            'ups',
            'downs',
            'gilded']

attributes_comments = ['id',
                        '_submission',
                       'subreddit_id',
                      'score',
                      'author',
                       'author_flair_text',
                       'archived',
                       'name',
                        'body',
                      'send_replies',
                    'distinguished',
                      'created',
                       'created_utc',
                    'approved_by',
                      'link_id',
                      'parent_id',
                      'ups',
                      'downs',
                      'gilded']

In [6]:
def attribute_to_df(df, obj, attribute, list_attributes):
    # Add attribute to the list
    att_value = getattr(obj, attribute)
    temp_list.append(att_value)
    
    # If all the attributes have been added to the list,
    # append the list to the dataframe
    if attribute == list_attributes[-1]:
        if len(df) > 0:
            df_temp = pd.DataFrame([temp_list])
            df = df.append(df_temp)
        if len(df) == 0:
            df = pd.DataFrame([temp_list])
    
    return df

In [7]:
num_subm = 1000
df = pd.DataFrame()
df2 = pd.DataFrame()
for idx, submission in enumerate(reddit.subreddit('CanadaPublicServants').hot(limit=num_subm)):
    for attribute in attributes_subm:
        # Initialize a list for each new object
        if attribute == attributes_subm[0]:
            temp_list = []
        df = attribute_to_df(df,submission,attribute,attributes_subm)
    
    for idx2, comment in enumerate(submission.comments):
        for attribute in attributes_comments:
            # Initialize a list for each new object
            if attribute == attributes_comments[0]:
                temp_list = []
            df2 = attribute_to_df(df2,comment,attribute,attributes_comments)

### Define column's name + column's index

In [8]:
'''
print(df.iloc[1].index)
print(df_submissions.iloc[1].index)

for i in range(0, len(df.columns)):
    print(df.columns[i])
    print(type(df_submissions.iloc[1][i]))
    print(type(df.iloc[1][i]))
    print(df_submissions.iloc[1][i])
    print(df.iloc[1][i])
'''

'\nprint(df.iloc[1].index)\nprint(df_submissions.iloc[1].index)\n\nfor i in range(0, len(df.columns)):\n    print(df.columns[i])\n    print(type(df_submissions.iloc[1][i]))\n    print(type(df.iloc[1][i]))\n    print(df_submissions.iloc[1][i])\n    print(df.iloc[1][i])\n'

In [9]:
df.columns = attributes_subm

df2.columns = attributes_comments
df2 = df2.rename(columns={'id':'comment_id'})

print(type(df)); print(df.shape); print(df.dtypes)
print(df[0:5])
print(type(df2)); print(df2.shape); print(df2.dtypes)
print(df2[0:5])

<class 'pandas.core.frame.DataFrame'>
(1000, 25)
id                    object
subreddit_id          object
title                 object
score                  int64
author                object
author_flair_text     object
likes                 object
archived                bool
is_video                bool
name                  object
over_18                 bool
pinned                  bool
quarantine              bool
selftext              object
send_replies            bool
num_comments           int64
num_crossposts         int64
distinguished         object
created              float64
created_utc          float64
comment_limit          int64
approved_by           object
ups                    int64
downs                  int64
gilded                 int64
dtype: object
       id subreddit_id                                              title  \
0  8lkb0l     t5_3f8vd  How flexible are teleworking options at your d...   
0  8ljbjv     t5_3f8vd  Manager wants to talk to my new ma

# Append new data points to current dataframe

In [10]:
new_submissions = pd.concat([df_submissions,df])
print(new_submissions.shape)
new_submissions = new_submissions.drop_duplicates(subset=['selftext'], keep='last')

new_comments = pd.concat([df_comments,df2])
print(new_comments.shape)
new_comments = new_comments.drop_duplicates(subset=['body'], keep='last')

print('Lenght of the old {}'.format(len(df_submissions))) 
print('Lenght of the new {}'.format(len(new_submissions))) 
print('Lenght of the old {}'.format(len(df_comments))) 
print('Lenght of the new {}'.format(len(new_comments))) 

print(df_submissions.shape), print(new_submissions.shape)
print(df_comments.shape), print(new_comments.shape)

print('{} new submissions added'.format(len(new_submissions)-len(df_submissions)))
print('{} new comments added'.format(len(new_comments)-len(df_comments)))

(2505, 25)
(12127, 19)
Lenght of the old 1505
Lenght of the new 2091
Lenght of the old 7555
Lenght of the new 9946
(1505, 25)
(2091, 25)
(7555, 19)
(9946, 19)
586 new submissions added
2391 new comments added


In [11]:
print(new_submissions.columns)
print(new_comments.columns)

Index(['id', 'subreddit_id', 'title', 'score', 'author', 'author_flair_text',
       'likes', 'archived', 'is_video', 'name', 'over_18', 'pinned',
       'quarantine', 'selftext', 'send_replies', 'num_comments',
       'num_crossposts', 'distinguished', 'created', 'created_utc',
       'comment_limit', 'approved_by', 'ups', 'downs', 'gilded'],
      dtype='object')
Index(['comment_id', '_submission', 'subreddit_id', 'score', 'author',
       'author_flair_text', 'archived', 'name', 'body', 'send_replies',
       'distinguished', 'created', 'created_utc', 'approved_by', 'link_id',
       'parent_id', 'ups', 'downs', 'gilded'],
      dtype='object')


# 3) Export Dataframes to CSV

In [12]:
import os
import sys
import datetime

now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
print(now)

print('reddit_comments_submission_' + str(now) + '.csv')

2018-05-23_1159
reddit_comments_submission_2018-05-23_1159.csv


In [13]:
new_submissions.to_csv('reddit_submissions_wUTC_' + str(now) + '.csv', index=True, header = True)
new_comments.to_csv('reddit_comments_wUTC_' + str(now) +'.csv', index=True, header = True)